In [4]:
import pandas as pd

pd.set_option('display.max_rows', None)  # Show all rows
pd.set_option('display.max_columns', None)  # Show all columns

pd.set_option('display.max_colwidth', None)  # No limit on column width


Target Data

In [64]:
import requests
import re
import pandas as pd
from urllib.parse import quote, unquote
from html import unescape
import json

# Endpoint and headers
url = "https://redsky.target.com/redsky_aggregations/v1/web/plp_search_v2?"
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36'}

# List of ingredients to be able to loop
ingredients = ["unsalted butter", "granulated sugar", "brown sugar", "eggs", "vanilla extract", "baking soda", "all purpose flour", "semi-sweet chocolate chip", "chopped walnuts"]

# Parameters for the API
parameters = {
    "key": "9f36aeafbe60771e321a7cc95a78140772ab3e96",
    "channel": "WEB",
    "count": 24,
    "default_purchasability_filter": "true",
    "include_dmc_dmr": "true",
    "include_sponsored": "true",
    "new_search": "true",
    "platform": "desktop",
    "pricing_store_id": 2455,
    "spellcheck": "true",
    "store_ids": [2455, 2268, 2408, 310, 3384],
    "visitor_id": "01939852639E02019551A1FC131A3326",
    "zip": 95616
}

def target_scraper(ingredient, regex_pattern):
    all_results = []
    offset = 0

    while True:
        # Update parameters dynamically for each request
        encoded_ingredient = quote(ingredient)
        parameters["keyword"] = ingredient
        parameters["page"] = f"/s/{encoded_ingredient}"
        parameters["offset"] = offset

        response = requests.get(url, params=parameters, headers=headers)
        
        # Check if response is successful
        if response.status_code != 200:
            print(f"Error fetching data for {ingredient}: {response.status_code}")
            break

        # Load response data
        try:
            data = response.json()
            products = data["data"]["search"]["products"]
        except (KeyError, json.JSONDecodeError):
            print(f"Error parsing data for {ingredient}")
            break
        
        # If no more products, exit loop
        if not products:
            break

        # Extract product name and price
        for product in products:
            title = product['item']['product_description']['title']
            price = product['price']['current_retail']
            decoded_title = unescape(unquote(title))

            # Filter products based on the regex pattern
            if re.search(regex_pattern, decoded_title, flags=re.IGNORECASE):
                all_results.append({'ingredient': ingredient, 'product_name': decoded_title, 'price': price})

        # Increment offset for next page
        offset += 24

    return pd.DataFrame(all_results)

# Define ingredient list and their filtering terms
filter_terms = {
    "unsalted butter": r"unsalted butter",
    "granulated sugar": r"granulated sugar",
    "brown sugar": r"brown sugar",
    "eggs": r"eggs",
    "vanilla extract": r"vanilla extract",
    "baking soda": r"baking soda",
    "all purpose flour": r"all purpose flour|all\-purpose flour",
    "semi-sweet chocolate chip": r"semi sweet chocolate|semi\-sweet chocolate",
    "chopped walnuts": r"chopped walnuts"
}

# Loop through each ingredient and collect data
main_df = pd.DataFrame()
for ingredient, regex_pattern in filter_terms.items():
    print(f"Scraping for: {ingredient}")
    ingredient_df = target_scraper(ingredient, regex_pattern)
    main_df = pd.concat([main_df, ingredient_df], ignore_index=True)

main_df.reset_index(drop=True, inplace=True)

def extract_amount(product_name):
    # Extracing the amount by using common units
    matches = re.findall(r"(\d+\.?\d*)\s?(fl\.?\s?oz|oz|lb|lbs|g|kg|ml|l|ct|pcs|pack|case|floz)", product_name, re.IGNORECASE)
    if matches:
        return f"{matches[-1][0]} {matches[-1][1].replace('.', '').lower()}"
    return None

# Creates an amount value for each of the products
main_df["amount"] = main_df["product_name"].apply(extract_amount)

# Remove rows with None in the "amount" column
main_df = main_df[main_df["amount"].notna()]

def filter_eggs(df):
    # Filtering out egg products that contain the word large, candy, or hard since they are not the type of eggs we are looking for
    return df[~((df["ingredient"] == "eggs") & 
                ((df["product_name"].str.contains("hard", case=False)) | 
                 (~df["product_name"].str.contains("large", case=False)) |
                 (df["product_name"].str.contains("candy", case=False))))]

filtered_df = filter_eggs(main_df)


Scraping for: unsalted butter
Scraping for: granulated sugar
Scraping for: brown sugar
Scraping for: eggs
Scraping for: vanilla extract
Scraping for: baking soda
Scraping for: all purpose flour
Scraping for: semi-sweet chocolate chip
Scraping for: chopped walnuts


In [66]:
# Filter for filtering out products that don't meet our minimum required amount per ingredient
FILTER_CRITERIA = {
    "unsalted butter": {"lb": 0.5, "ct": 2},
    "granulated sugar": {"lb": 0.5},
    "brown sugar": {"lb": 0.5, "oz": 7},
    "eggs": {"ct": 2},
    "all purpose flour": {"lb": 1, "g": 120, "oz": 4},
    "semi-sweet chocolate chip": {"oz": 12},
    "chopped walnuts": {"oz": 6},
    # Baking soda and vanilla extract will not need filtering as all products have enough (1-2 tablespoons)
}

def meets_criteria(row):
    # Filtering out products that dont meet the filtering criteria
    ingredient = row["ingredient"]
    
    # If the ingredient is baking soda or vanilla extract, keep it as is as these products have enough
    if ingredient in ["baking soda", "vanilla extract"]:
        return True

    amount_unit = row["amount"].split()
    
    if len(amount_unit) != 2:
        return False
    
    amount, unit = amount_unit
    amount = float(amount)
    
    # Check if the unit and amount match the criteria for this ingredient
    if ingredient in FILTER_CRITERIA and unit in FILTER_CRITERIA[ingredient]:
        return amount >= FILTER_CRITERIA[ingredient][unit]
    
    return False

filtered_df = filtered_df[filtered_df.apply(meets_criteria, axis=1)]

filtered_df.reset_index(drop=True, inplace=True)

filtered_df

,ingredient,product_name,price,amount
0,unsalted butter,Unsalted Butter - 1lb - Good & Gather™,4.49,1 lb
1,unsalted butter,Clover Sonoma Unsalted Butter - 16oz/4ct Sticks,7.69,4 ct
2,granulated sugar,Granulated Sugar - 4lbs - Good & Gather™,3.39,4 lb
3,granulated sugar,C&H Premium Pure Cane Granulated Sugar - 4lbs,4.19,4 lb
4,granulated sugar,Domino Granulated Sugar - 3.5 Lb,21.99,3.5 lb
5,brown sugar,Light Brown Sugar - 2lbs - Good & Gather™,1.99,2 lb
6,brown sugar,Dark Brown Sugar - 2lbs - Good & Gather™,1.99,2 lb
7,brown sugar,C&H Premium Pure Cane Light Brown Sugar - 2lbs,4.29,2 lb
8,brown sugar,C&H Premium Pure Cane Dark Brown Sugar - 2lbs,4.29,2 lb
9,brown sugar,Organic Light Brown Sugar - 24oz - Good & Gather™,3.69,24 oz


In [ ]:
import pandas as pd

filtered_df = filtered_df.copy()

# Special handling for "brown sugar" preference
if "brown sugar" in filtered_df['ingredient'].values:
    # Create an "is_dark" column for brown sugar products
    filtered_df.loc[filtered_df['ingredient'] == "brown sugar", 'is_dark'] = \
        filtered_df['product_name'].str.contains(r"\bdark brown sugar\b", case=False, na=False, regex=True)

# Sort the DataFrame to prioritize dark brown sugar and lowest price for each ingredient
filtered_df.sort_values(by=['ingredient', 'is_dark', 'price'], ascending=[True, False, True], inplace=True)

# Drop duplicates to keep only the cheapest product per unique ingredient
cheapest_ingredients = filtered_df.drop_duplicates(subset='ingredient', keep='first')

cheapest_ingredients.reset_index(drop=True, inplace=True)

print(cheapest_ingredients[["ingredient","product_name","price","amount"]])


Davis Co-op Data

In [82]:
import requests
import re
import pandas as pd
from urllib.parse import quote, unquote
from html import unescape
import json

# Endpoint URL and header
url = "https://daviscoop.storebyweb.com/s/1000-1/api/b/"
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.3"}

ingredients = ["unsalted butter", "granulated sugar", "brown sugar", "eggs", "vanilla extract", "baking soda", "all purpose flour", "semi-sweet chocolate chip", "walnuts"]

parameters = {"facets": {}, "ps": 32, "s": "", "g": []}

def coop_scraper(ingredient, regex_pattern):
    all_results = []
    pn = 1

    while True:
        parameters["pn"] = pn
        parameters["q"] = ingredient

        response = requests.post(url, json=parameters, headers=headers)
        
        # Check if response is successful
        if response.status_code != 200:
            print(f"Error fetching data for {ingredient}: {response.status_code}")
            break

        # Load response data
        try:
            data = response.json()
            products = data.get("items",[])
        except (KeyError, json.JSONDecodeError):
            print(f"Error parsing data for {ingredient}")
            break
        
        # If no more products, exit loop
        if not products:
            break

        # Extract product name and price
        for product in products:
            title = product['name']
            price = product['actualPrice']
            amount = product['size']
            brand = product['brand']
            decoded_title = unescape(unquote(title))

            # Filter products based on the regex pattern
            if re.search(regex_pattern, decoded_title, flags=re.IGNORECASE):
                all_results.append({'ingredient': ingredient, 'product_name': decoded_title, 'price': price, 'amount': amount, "brand": brand})

        if len(products) < parameters["ps"]:
            break

        # Increment offset for next page
        pn += 1
    
    return pd.DataFrame(all_results)

# Define ingredient list and their filtering terms
filter_terms = {
    "unsalted butter": r"unsalted butter",
    "granulated sugar": r"granulated sugar",
    "brown sugar": r"brown sugar",
    "eggs": r"eggs",
    "vanilla extract": r"vanilla extract",
    "baking soda": r"baking soda",
    "all purpose flour": r"all purpose flour|all\-purpose flour",
    "semi-sweet chocolate chip": r"semi sweet chocolate|semi\-sweet chocolate",
    "walnuts": r"walnuts"
}

main2_df = pd.DataFrame()
for ingredient, regex_pattern in filter_terms.items():
    print(f"Scraping for: {ingredient}")
    ingredient_df = coop_scraper(ingredient, regex_pattern)
    main2_df = pd.concat([main2_df, ingredient_df], ignore_index=True)

main2_df.reset_index(drop=True, inplace=True)

def filter_eggs(df):
    # Exclude products that contain words that signal on egg products we are not looking for
    return df[~((df["ingredient"] == "eggs") & 
                ((df["product_name"].str.contains("hard", case=False)) | 
                 (~df["product_name"].str.contains("large", case=False)) |
                 (df["product_name"].str.contains("candy", case=False))))]

def filter_brown_sugar(df):
    # Exclude products that contain keywords like wafels that we are not looking for
    return df[~((df["ingredient"] == "brown sugar") & 
                ((df["product_name"].str.contains("creamer", case=False)) | 
                 (df["product_name"].str.contains("maple", case=False)) |
                 (df["product_name"].str.contains("replacement", case=False)) |
                 (df["product_name"].str.contains("wafels", case=False)) |
                 (df["product_name"].str.contains("ice", case=False)) |
                 (df["product_name"].str.contains("milk", case=False)) |
                 (df["product_name"].str.contains("latte", case=False))))]

def filter_walnuts(df):
    # Excluding walnut products from sprouted as they are not the type of walnuts we are looking for
    return df[~((df["ingredient"] == "walnuts") & 
                ((df["product_name"].str.contains("sprouted", case=False))))]

filtered_df2 = filter_eggs(main2_df)
filtered_df2 = filter_brown_sugar(filtered_df2)
filtered_df2 = filter_walnuts(filtered_df2)

Scraping for: unsalted butter
Scraping for: granulated sugar
Scraping for: brown sugar
Scraping for: eggs
Scraping for: vanilla extract
Scraping for: baking soda
Scraping for: all purpose flour
Scraping for: semi-sweet chocolate chip
Scraping for: walnuts


In [84]:
# Filter for filtering out products that don't meet our minimum required amount per ingredient
FILTER_CRITERIA = {
    "unsalted butter": {"oz.": 8},
    "granulated sugar": {"oz.": 8},
    "brown sugar": {"lb.": 0.5, "oz.": 7},
    "eggs": {"ct.": 2},
    "all purpose flour": {"lb.": 1, "g.": 120, "oz.": 4},
    "semi-sweet chocolate chip": {"oz.": 12},
    "walnuts": {"oz.": 6},
    # Baking soda and vanilla extract will not need filtering as all products have enough (1-2 tablespoons)
}

def meets_criteria(row):
    # Filtering out products that dont meet the filtering criteria
    ingredient = row["ingredient"]
    
    # If the ingredient is baking soda or vanilla extract, keep it as is as these products have enough
    if ingredient in ["baking soda", "vanilla extract"]:
        return True

    amount_unit = row["amount"].split()
    
    if len(amount_unit) != 2:
        return False
    
    amount, unit = amount_unit
    amount = float(amount)
    
    # Check if the unit and amount match the criteria for this ingredient
    if ingredient in FILTER_CRITERIA and unit in FILTER_CRITERIA[ingredient]:
        return amount >= FILTER_CRITERIA[ingredient][unit]
    
    return False

def preprocess_amount_column(df):
    # Some rows have invalid format so clean out the data frame to be able to run the filtering code by making sure every value in "amount" is a valid format
    cleaned_rows = []
    for index, row in df.iterrows():
        amount_unit = row["amount"].split()
        if len(amount_unit) == 2:  # Only keep rows with exactly two parts: number and unit
            try:
                float(amount_unit[0])
                cleaned_rows.append(row)
            except ValueError:
                continue
    return pd.DataFrame(cleaned_rows)

# Clean the 'amount' column
filtered_df2 = preprocess_amount_column(filtered_df2)

filtered_df2 = filtered_df2[filtered_df2.apply(meets_criteria, axis=1)]

filtered_df2.reset_index(drop=True, inplace=True)

filtered_df2

,ingredient,product_name,price,amount,brand
0,unsalted butter,Unsalted Butter,4.49,8 oz.,Vital Farms
1,unsalted butter,Unsalted Butter,10.49,16 oz.,Organic Valley
2,unsalted butter,Unsalted Butter,6.49,16 oz.,Tillamook
3,unsalted butter,Unsalted Butter,5.79,16 oz.,Crystal Creamery
4,unsalted butter,Unsalted Butter Quarters,6.99,16 oz.,Clover
5,unsalted butter,Unsalted Butter,8.49,16 oz.,Challenge Dairy Products
6,unsalted butter,Organic European Unsalted Butter,5.99,8 oz.,Rumiano Cheese
7,unsalted butter,French Unsalted Butter,6.99,8.8 oz.,Isigny
8,unsalted butter,European Unsalted Butter,7.99,16 oz.,Cremerie Classique
9,granulated sugar,Granulated Sugar,6.79,64 oz.,C&H


In [ ]:
import pandas as pd

# Ensure a clean DataFrame
filtered_df2 = filtered_df2.copy()

# Special handling for "brown sugar" preference
if "brown sugar" in filtered_df2['ingredient'].values:
    # Create an "is_dark" column for brown sugar products
    filtered_df2.loc[filtered_df2['ingredient'] == "brown sugar", 'is_dark'] = \
        filtered_df2['product_name'].str.contains(r"\bdark brown sugar\b", case=False, na=False, regex=True)

# Sort the DataFrame to prioritize dark brown sugar and lowest price for each ingredient
filtered_df2.sort_values(by=['ingredient', 'is_dark', 'price'], ascending=[True, False, True], inplace=True)

# Drop duplicates to keep only the cheapest product per unique ingredient
cheapest_ingredients = filtered_df2.drop_duplicates(subset='ingredient', keep='first')

cheapest_ingredients.reset_index(drop=True, inplace=True)

print(cheapest_ingredients)